In [1]:
from nanotag.data import NanotagData, ImageFileCollection, Summary
from nanotag.image import ImageSeries, GaussianFilterSlider
from nanotag.tags import PointTags, PointTagSeries
from nanotag.timeline import Timeline, TimelineTags, EditTimelineTags
from nanotag.canvas import Canvas, ToolBox
from nanotag.tools import EditPointTags, ResetView, BoxZoomTool, PanZoomTool
from nanotag.utils import link
from nanotag.histogram import Histogram
from traitlets import directional_link

from bqplot import ColorScale, Scatter, Figure
import ipywidgets as widgets
import os
import numpy as np

from IPython.core.display import display, HTML
display(HTML("<style>.container { width:80% !important; }</style>"))

In [2]:
root_directory = '/data/'
#root_directory = 'D:/data/julia/'
analysis_file = 'ml_analysis.json'
#filt = '/data/*.tif'
filt = '**/*.tif'

In [3]:
atom_tags = PointTagSeries(data_fields=('sublattice', 'intensities', 'mask'), enable_move=False)
link((atom_tags.point_tags, 'mask'), (atom_tags.point_tags._mark, 'opacity'))

defect_tags = PointTagSeries(data_fields=('type',))
area_tags = TimelineTags(row='area', data_fields=('area',), enable_move=False, color_scale=ColorScale(min=0, max=1))

image_series = ImageSeries()
gaussian_filter = GaussianFilterSlider(max=10)
image_series.filters = [gaussian_filter]

nanotag_data = NanotagData(root_directory=root_directory,
                           read_file = analysis_file,
                           write_file = analysis_file,
                           tags = {'points' : atom_tags, 'defects': defect_tags, 'area': area_tags}
                           )
image_file_collection = ImageFileCollection(filter=filt, image_series=image_series)
link((nanotag_data, 'root_directory'), (image_file_collection, 'root_directory'))
link((image_file_collection, 'hash'), (nanotag_data, 'identifier'), check_broken=False)


canvas = Canvas()
canvas.tags = [atom_tags, defect_tags]
canvas.image = image_series


timeline = Timeline()
timeline.tags = [area_tags]
link((image_series, 'num_frames'), (timeline, 'num_frames'))
directional_link((timeline, 'frame_index'), (image_series, 'frame_index'))
directional_link((timeline, 'frame_index'), (atom_tags, 'frame_index'))
directional_link((timeline, 'frame_index'), (defect_tags, 'frame_index'))

histogram1 = Histogram(markers=('S','S2'), height=220, min=.0, max=1.5, bins=30, label='S sublattice', adjust_x_scale=False)
histogram2 = Histogram(markers=('Mo',), height=220, min=.0, max=3, bins=30, label='Mo sublattice', adjust_x_scale=False)

histogram_checkbox = widgets.Checkbox(description='Normalized')

toolbox = ToolBox(canvas)
toolbox.tools = {'Reset' : ResetView(),
                 'BoxZoom' : BoxZoomTool(),
                 'Pan' : PanZoomTool(),
                 'S defect' : EditPointTags(defect_tags, data_fields={'type':0}),
                 'Mo defect' : EditPointTags(defect_tags, data_fields={'type':1}),
                }

summary = Summary(write_file='summary.json')

In [4]:
def create_frame_summary(i):
    summary = {}
    summary['frame_index'] = i
    summary['lattice_area_fraction'] = 0.
    summary['defect_type'] = []
    summary['defect_x'] = []
    summary['defect_y'] = []
    
    if len(area_tags.area) == 0:
        return summary
    
    summary['lattice_area_fraction'] = area_tags.area[i]
    
    try:
        summary['sampling'] = nanotag_data.selected_data['sampling'][str(i)]
    except KeyError:
        summary['sampling'] = None
    
    if not i in defect_tags.series.keys():
        return summary
    
    types = defect_tags.series[i]['type']
    mapping = {0:'S2',1:'S',2:'Mo'}
    
    summary['defect_type'] = [mapping[t] for t in types]
    summary['defect_x'] = defect_tags.series[i]['x']
    summary['defect_y'] = defect_tags.series[i]['y']
    
    return summary

def create_series_summary():
    summary = {}
    summary['md5'] = image_file_collection.hash
    summary['num_frames'] = image_series.num_frames
    summary['shape_x'] = image_series.images.shape[1]
    summary['shape_y'] = image_series.images.shape[2]
    
    summary['boundary'] = 3.15 / np.sqrt(3)
    summary['normalized_S2_threshold'] = histogram1.S2
    summary['normalized_S_threshold'] = histogram1.S
    summary['normalized_Mo_threshold'] = histogram2.Mo
    summary['path'] = image_file_collection.relative_path
    return summary

def update_summary():
    current_data = create_series_summary()
    current_data.update(create_frame_summary(timeline.frame_index))
    
    data = create_series_summary()
    for i in range(len(defect_tags.series)):
        data[i] = create_frame_summary(i)
    
    summary.current_data = current_data
    summary.data = data

defect_tags.point_tags.observe(lambda *args: update_summary(), 'x')
    
def update_histograms():
    intensities1 = atom_tags.point_tags.intensities[(atom_tags.point_tags.sublattice == 0) * (atom_tags.point_tags.mask == 1)]
    intensities2 = atom_tags.point_tags.intensities[(atom_tags.point_tags.sublattice == 1) * (atom_tags.point_tags.mask == 1)]
    
    if len(intensities1) == 0:
        mean1 = 1
        mean2 = 1
    else:
        mean1 = np.abs(intensities1.mean())
        mean2 = np.abs(intensities2.mean())
    
    if histogram_checkbox.value:
        intensities1 = intensities1 / mean1
        intensities2 = intensities2 / mean2
    
    histogram1.sample = intensities1
    histogram2.sample = intensities2
    
    return mean1, mean2

def update_histogram_checkbox(change):
    mean1, mean2 = update_histograms()
    
    if change['new']:
        histogram1.min=0
        histogram1.max=1.5
        histogram2.min=0
        histogram2.max=1.5
        
        histogram1.S = histogram1.S / mean1
        histogram1.S2 = histogram1.S2 / mean1
        histogram2.Mo = histogram2.Mo / mean2
    else:
        histogram1.min=0
        histogram1.max=2.5
        histogram2.min=0
        histogram2.max=2.5
        
        histogram1.S = histogram1.S * mean1
        histogram1.S2 = histogram1.S2 * mean1
        histogram2.Mo = histogram2.Mo * mean2
    
    
atom_tags.point_tags.observe(lambda *args: update_histograms(), 'intensities')

 
histogram_checkbox.observe(update_histogram_checkbox, 'value')
histogram_checkbox.value = True

def area_changed(change):
    if len(area_tags.area) > 0:
        timeline.frame_index = int(np.argmax(area_tags.area > .5))

area_tags.observe(area_changed, 'area')

def defect_threshold_changed():
    series = {}
    for key, value in atom_tags.series.items():
        series[key] = {'x':[], 'y':[], 'type':[]}
        
        if area_tags.area[key] < .5:
            continue
        
        mask = np.array(value['mask'], dtype=bool)
        x = np.array(value['x'])
        y = np.array(value['y'])
        sublattice = np.array(value['sublattice'])
        intensities = np.array(value['intensities'])
        
        intensities1 = intensities[(sublattice == 0) * (mask == 1)]
        intensities2 = intensities[(sublattice == 1) * (mask == 1)]
        
        if histogram_checkbox.value:
            intensities1 = intensities1 / np.abs(intensities1.mean())
            intensities2 = intensities2 / np.abs(intensities2.mean())
        
        S2_thres = min(histogram1.S2, histogram1.S)
        S_thres = max(histogram1.S2, histogram1.S)
        
        S2 = (intensities1 < S2_thres)
        S = (intensities1 < S_thres) * (intensities1 > S2_thres)
        Mo = (intensities2 < histogram2.Mo)
        
        
        x = np.concatenate((x[(sublattice == 0) * (mask == 1)][S2], 
                            x[(sublattice == 0) * (mask == 1)][S],
                            x[(sublattice == 1) * (mask == 1)][Mo]))
        
        y = np.concatenate((y[(sublattice == 0) * (mask == 1)][S2], 
                            y[(sublattice == 0) * (mask == 1)][S],
                            y[(sublattice == 1) * (mask == 1)][Mo]
                           ))
        
        if len(x) == 0:
            continue
        
        types = [0] * sum(S2) + [1] * sum(S) + [2] * sum(Mo)
        
        series[key]['x'] = x.tolist()
        series[key]['y'] = y.tolist()
        series[key]['type'] = types
    
    defect_tags.series = series

histogram1.observe(lambda *args: defect_threshold_changed(), 'S')
histogram1.observe(lambda *args: defect_threshold_changed(), 'S2')
histogram2.observe(lambda *args: defect_threshold_changed(), 'Mo')
nanotag_data.observe(lambda *args: defect_threshold_changed(), 'identifier')

def transform_summary_path(x):
    if x is None:
        return ''
    
    return os.path.splitext(x)[0] + '.json'

def transform_frame_summary_path(x):
    if x is None:
        return ''
    
    return os.path.splitext(x)[0] + f'_{timeline.frame_index:03d}.json'

def set_current_path():
    summary.current_write_file = transform_frame_summary_path(image_file_collection.path)

timeline.observe(lambda *args: set_current_path() , 'frame_index')
directional_link((image_file_collection, 'path'), (summary, 'current_write_file'), transform_frame_summary_path)
directional_link((image_file_collection, 'path'), (summary, 'write_file'), transform_summary_path);

In [5]:
nanotag_data.read_data()
defect_threshold_changed()

toolbox.button_width = '87px'
atom_tags.point_tags.color_scheme = 'plasma'
defect_tags.point_tags.color_scheme = 'category'
defect_tags.point_tags.color_scale.min=0
defect_tags.point_tags.color_scale.max=10

tools = widgets.VBox([nanotag_data, image_file_collection, image_series, gaussian_filter, atom_tags, defect_tags, toolbox])
hisogram_box = widgets.VBox([histogram1, histogram2, histogram_checkbox])

tabs = widgets.Tab([tools, hisogram_box])
tabs.set_title(0, 'Tools')
tabs.set_title(1, 'Histograms')

canvas_box = widgets.VBox([canvas, timeline])

app = widgets.HBox([canvas_box, tabs, summary])
app

FileNotFoundError: [Errno 2] No such file or directory: '/data/ml_analysis.json'

In [44]:
from nanotag.events import KeyEvents

events = KeyEvents(canvas_box)

def toggle_point_visible(*args):
    atom_tags.point_tags.visible = not atom_tags.point_tags.visible

def toggle_defects_visible(*args):
    defect_tags.point_tags.visible = not defect_tags.point_tags.visible

    
events.callbacks = {'ArrowLeft' : timeline.previous_frame,
                    'ArrowRight' : timeline.next_frame,
                    'ArrowUp' : image_file_collection.next_file,
                    'ArrowDown' : image_file_collection.previous_file,
                    'r' : canvas.reset,
                    'z' : lambda : toolbox.toggle_tool('BoxZoom'),
                    'p' : lambda : toolbox.toggle_tool('Pan'),
                    'l' : toggle_point_visible,
                    'd' : toggle_defects_visible,
                   }